# Assigning super_book_codes to the banned books

**Author:** Michael Falk

**Date:** 31/10/18-1/11/18

## Background

One of the key datasets for *Mapping Print, Charting Enlightenment* is a set of documents concerning illegal books in eighteenth-century France. BNF MS 21928-9 contains a list of banned books. It is unclear who exactly wrote the list, but it appears to have been prepared by the central government to assist book inspectors with their tasks across France. BNF Arsenal MS 10305 is an inventory of all the books that were found in the Bastille when it was stormed during the French Revolution. The actual MS has disappeared, but luckily a modern edition exists.

A problem occured during entry of the data. The interface was supposed to oblige the user to assign a 'super book code' to each title in the banned books lists upon entry. But due to a glitch in the interface, the lookup took too long at it was impossible to efficiently do so. Accordingly, only 97 of the 1000+ illegal books have a 'super book code' assigned to them. The data is therefore not linked to the rest of the database and is useless for analysis.

To speed up the process of linking all the data, this notebook uses 'dedupe', an open-source record linkage library, to try and find links between these banned titles and the titles already recorded elsewhere in the database. Hopefully this will speed up record linkage, and provide a testbed for other record linkage tasks in the project.

***

## Section 1: Import data, initialise model

In [74]:
# Cell 1.1: Load necessary libraries and define key paths
import dedupe as dd
import pandas as pd
import os as os
import time
import numpy as np
import random

input_file = "combined_editions_illegal_books.csv"
output_file = "illegal_books_deduped.csv"
settings_file = "illegal_books_learned_settings"
training_file = "illegal_books_training.json"
output_file = "illegal_books_clustered.csv"

Data was preprocessed in R. The full set of 'editions' was extracted from the database. The illegal_titles data was cleaned, and the two datasets were combined into a single large table. The script is in this repo. This preprocessing means that the problem is now a problem of finding duplicate rows in a single table.

In [69]:
# Cell 1.2: Import data
data_frame = pd.read_csv(input_file)

print(f"The data has {data_frame.shape[0]} rows, {data_frame[data_frame['super_book_code'].isna()].shape[0]} of which need super_book_codes assigned.")

# Dedupe requires missing values to be 'Nones', and wants the data to be a list of dicts.
data_frame = data_frame.where(pd.notnull(data_frame), None)
data = data_frame.to_dict("index")

The data has 14201 rows, 1941 of which need super_book_codes assigned.


In [3]:
# Cell 1.3: Define initialisation function
def dedupe_initialise(data, fields, settings_file, training_file, sample_size = 15000):
    """
    Takes a data dictionary and field definitions and creates a Dedupe object.
    
    params:
        data: a list of dicts, where each dict represents one record
        fields: a list of dicts, where each dict describes a field that the model should inspect
    
    returns:
        deduper: a Dedupe object
    """
    
    # Check to see if an initialised model has already been saved.
    # If so, load the initialised model. If not, initialise a new model using the fields list supplied.
    if os.path.exists(settings_file):
        print('reading from', settings_file)
        with open(settings_file, 'rb') as f:
            deduper = dedupe.StaticDedupe(f)
    else:
        deduper = dd.Dedupe(fields)
    
    # Create training sample pairs from provided data
    deduper.sample(data, sample_size)
    
    # Load existing training file if it exists
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file, 'rb') as f:
            deduper.readTraining(f)
    
    return deduper

In [4]:
# Cell 1.4: Initialise deduper

# Creat a list of fields for the model to look at. NB: 'ID' and 'UUID' are not relevant to the task,
# hence do not appear in the list. The 'super_book_code' also encodes no useful information,
# because the problem is that we have records without codes, and the model will learn to focus on that
# column too much if we include it, since it is a perfect determinant of identity.
fields = [
    {'field':'full_book_title', 'type': 'String'},
    {'field':'author_name', 'type': 'String'},
    {'field':'stated_publication_places', 'type': 'String'},
    {'field':'stated_publication_years', 'type': 'DateTime'}
]

deduper = dedupe_initialise(data, fields, settings_file, training_file)

INFO:dedupe.canopy_index:Removing stop word  L
INFO:dedupe.canopy_index:Removing stop word  d
INFO:dedupe.canopy_index:Removing stop word  p
INFO:dedupe.canopy_index:Removing stop word a 
INFO:dedupe.canopy_index:Removing stop word du
INFO:dedupe.canopy_index:Removing stop word em
INFO:dedupe.canopy_index:Removing stop word es
INFO:dedupe.canopy_index:Removing stop word hi
INFO:dedupe.canopy_index:Removing stop word is
INFO:dedupe.canopy_index:Removing stop word mo
INFO:dedupe.canopy_index:Removing stop word oi
INFO:dedupe.canopy_index:Removing stop word po
INFO:dedupe.canopy_index:Removing stop word re
INFO:dedupe.canopy_index:Removing stop word s 
INFO:dedupe.canopy_index:Removing stop word st
INFO:dedupe.canopy_index:Removing stop word u 
INFO:dedupe.canopy_index:Removing stop word ur
INFO:dedupe.canopy_index:Removing stop word  P
INFO:dedupe.canopy_index:Removing stop word  e
INFO:dedupe.canopy_index:Removing stop word Co
INFO:dedupe.canopy_index:Removing stop word ab
INFO:dedupe.c

## Section 2: Training the Model

In [6]:
# Cell 2.1: Collecting training data

# Running this cell will open an interactive shell in the output below, where you will be
# presented with pairs of books, and will be asked to say if they are the same or different.
dd.consoleLabel(deduper)

full_book_title : Bibliotheque des sciences et des beaux arts
author_name : Joncourt, Elie [de]
stated_publication_places : La Haye
stated_publication_years : None

full_book_title : Heures dédiées à monseigneur le dauphin
author_name : None
stated_publication_places : Bayeux
stated_publication_years : 1784

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


n


full_book_title : Exposition historique et apologetique des miracles perpetuels
author_name : None
stated_publication_places : None
stated_publication_years : 1707

full_book_title : Nouveau Dictionnaire (le) Suisse, François-Allemand, et Allemand-François
author_name : Poetevin, Franz Ludwig
stated_publication_places : None
stated_publication_years : None

0/10 positive, 1/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


full_book_title : Oeuvres de Regnard
author_name : Regnard, Jean François
stated_publication_places : Paris
stated_publication_years : 1788

full_book_title : Synonymes françois
author_name : Girard, Gabriel
stated_publication_places : Rouen
stated_publication_years : 1788

0/10 positive, 2/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

full_book_title : Considérations sur les causes phisiques et morales de la diversité du geenie, des moeurs et du gouvernement des nations: tiré en partie d'un ouvrage anonime; par M. L. Castilhon
author_name : Castilhon, Jean-Louis
stated_publication_places : Bouillon
stated_publication_years : 1769

full_book_title : Essais de Philosophie et de morale en partie traduit librement et en partie imite de Plutarque par M. L. Castilhon
author_name : Castilhon, Jean-Louis
stated_publication_places : Bouillon
stated_publication_years : 1770

20/10 positive, 59/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


full_book_title : Le Livre des Comptes faits, ou l’on trouve les supputations qui se font par les multiplications pour la valeur de quelque chose que l’on puisse s’imaginer, à telle somme qu’elles puissent monter etc. augmenté du tarif des glaces, par Bareme
author_name : Barrême, François Bertrand de
stated_publication_places : Paris
stated_publication_years : 1770

full_book_title : Comptes faits ou tarif général des monnoies
author_name : Barrême, François Bertrand de
stated_publication_places : Amsterdam
stated_publication_years : 1774

20/10 positive, 60/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling


In [10]:
# Cell 2.2: A function for training the model with the collected data and saving the results.
def run_deduper(deduper, data, settings_file, training_file, recall_weight = 1):
    """
    Given a deduper object and a dataset, this function trains the model and
    predicts which records are duplicates.
    
    params:
        deduper: a Dedupe object
        data: a dict of table rows/database records
        settings_file: a string giving the path where the settings should be written
        training_file: a string giving the path where the labelled training examples should be written
        recall_weight: a number indicating how much to privilege recall over precision.
        
    returns:
        deduper: the trained Dedupe object
        matches: a list of tuples giving record ids of duplicates and confidence scores
    """
    
    # Train the model
    print("Training model...")
    start = time.perf_counter()
    deduper.train()
    end = time.perf_counter()
    print(f"Training complete. It took {end - start:.3f} seconds.")
    
    
    print("Saving training data and trained parameters...")
    # Save the training examples
    with open(training_file, 'w') as tf:
        deduper.writeTraining(tf)
    
    # Save the model parameters
    with open(settings_file, 'wb') as sf:
        deduper.writeSettings(sf)
    print("Saved.")
    
    # Calculate threshold for matches
    print(f"Computing threshold based on a recall weighting of {recall_weight}.")
    start = time.perf_counter()
    threshold = deduper.threshold(data, recall_weight = 1)
    end = time.perf_counter()
    print(f"Computation complete. Threshold = {threshold}. It took {end - start:.3f} seconds.")
    
    # Compute the matches
    print("Clustering...")
    start = time.perf_counter()
    matches = deduper.match(data, threshold)
    end = time.perf_counter()
    print(f"Clustering complete. {len(matches)} clusters found. It took {end - start:.3f} seconds.")
    
    return deduper, matches

In [11]:
# Cell 2.3: Run the model, check out the results.
deduper, matches = run_deduper(deduper, data, settings_file, training_file, recall_weight = 1)

Training model...


INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.000010, score 0.5242863079293838
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sameSevenCharStartPredicate, author_name), TfidfNGramCanopyPredicate: (0.6, author_name))
INFO:dedupe.training:(SimplePredicate: (wholeFieldPredicate, full_book_title), TfidfTextCanopyPredicate: (0.6, full_book_title))


Training complete. It took 1.028 seconds.


INFO:dedupe.canopy_index:Removing stop word ar
INFO:dedupe.canopy_index:Removing stop word er
INFO:dedupe.canopy_index:Removing stop word ie
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word  J
INFO:dedupe.canopy_index:Removing stop word is
INFO:dedupe.canopy_index:Removing stop word e 
INFO:dedupe.canopy_index:Removing stop word du
INFO:dedupe.canopy_index:Removing stop word sur
INFO:dedupe.canopy_index:Removing stop word de
INFO:dedupe.canopy_index:Removing stop word par
INFO:dedupe.canopy_index:Removing stop word le
INFO:dedupe.canopy_index:Removing stop word des
INFO:dedupe.canopy_index:Removing stop word les
INFO:dedupe.canopy_index:Removing stop word et
INFO:dedupe.canopy_index:Removing stop word ou
INFO:dedupe.canopy_index:Removing stop word M
INFO:dedupe.blocking:10000, 2.1993132 seconds
INFO:dedupe.api:Maximum expected recall and precision
INFO:dedupe.api:recall: 0.796
INFO:dedupe.api:precision: 0.726
INFO:dedupe.api:With threshold: 0.3

Clustering...


INFO:dedupe.canopy_index:Removing stop word ar
INFO:dedupe.canopy_index:Removing stop word er
INFO:dedupe.canopy_index:Removing stop word ie
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word  J
INFO:dedupe.canopy_index:Removing stop word is
INFO:dedupe.canopy_index:Removing stop word e 
INFO:dedupe.canopy_index:Removing stop word du
INFO:dedupe.canopy_index:Removing stop word sur
INFO:dedupe.canopy_index:Removing stop word de
INFO:dedupe.canopy_index:Removing stop word par
INFO:dedupe.canopy_index:Removing stop word le
INFO:dedupe.canopy_index:Removing stop word des
INFO:dedupe.canopy_index:Removing stop word les
INFO:dedupe.canopy_index:Removing stop word et
INFO:dedupe.canopy_index:Removing stop word ou
INFO:dedupe.canopy_index:Removing stop word M
INFO:dedupe.blocking:10000, 2.0055242 seconds


Clustering complete. 2807 clusters found. It took 39.278 seconds.


In [130]:
# Cell 2.4: Adds cluster codes and confidence scores back to original data frame
def save_clusters(matches, data_frame, output_file):
    """
    Given a list of cluster tuples from a Dedupe object, and the original data frame
    on which the model was trained, this function outputs a data frame and saves a csv
    of the cluster assignments for each record/row.
    
    depends:
        pandas
        dedupe
    
    params:
        matches: a list of tuples, returned by Dedupe.matches()
        data_frame: the original data frame on which the model was trained.
        output_file: a string; the path where the csv will be written
    
    returns:
        data_frame: the original data frame will additional information from the model
    """
    
    # Add new columns to data frame
    data_frame['cluster'] = np.nan
    data_frame['confidence'] = np.nan
    
    # Loop through matches, update relevant rows
    for counter, match in enumerate(matches):
        data_frame.loc[match[0], 'cluster'] = int(counter)
        data_frame.loc[match[0], 'confidence'] = match[1]
    
    # Write csv
    with open(output_file, 'w') as out:
        data_frame.to_csv(out)
    
    return data_frame

In [131]:
# Cell 2.5. Invoke new function to update and export the results.
data_frame_clustered = save_clusters(matches, data_frame, output_file)

In [134]:
# Cell 2.6. Sanity check. How good are the model's assignments?
# Run this cell a few times to look at different random clusters
data_frame_clustered[data_frame_clustered['cluster'] == random.randint(0, len(matches) + 1)]

,ID,UUID,super_book_code,full_book_title,author_code,author_name,stated_publication_places,stated_publication_years,cluster,confidence
4059,None,None,spbk0000419,"Contes très mogols, enrichis de notes, avis, a...",au0000801,"Saint-Just, Simon-Pierre Mérard [de]",None,None,1038.0,0.787093
10084,None,None,spbk0000419,Contes très mogols,au0000801,"Saint-Just, Simon-Pierre Mérard [de]",Geneva,1770,1038.0,0.787093


In [141]:
# Cell 2.7. How much time have we saved? How many illegal books have been assigned a super_book_code?
assigned = data_frame_clustered[
    pd.notnull(data_frame_clustered['cluster']) & # which books have been assigned a cluster?
    pd.notnull(data_frame_clustered['UUID']) & # only illegal books have UUIDs
    pd.isna(data_frame_clustered['super_book_code']) # only interested in books that didn't already have super_book_codes
].shape[0]

total = data_frame_clustered[
    pd.notnull(data_frame_clustered['UUID']) &
    pd.isna(data_frame_clustered['super_book_code'])
].shape[0]

print(f"{assigned} illegal books have been given super_book_codes, of {total} that lack them.")

317 illegal books have been given super_book_codes, of 1921 that lack them.


## Conclusion

Dedupe appears to do a good job in linking the illegal books to super books that are already in the dataset. The question is whether the ~1600 books the algorithm could not cluster are new books that aren't already in our dataset, or if the algorithm has low recall.

Manual investigation will be necessary to see if the ~1600 super books are already in the database.

It may also be possible to tune the model further, by
1. giving it more training data, or
2. increasing the `recall_weight` so that the model cares more about finding possible matches than being accurate when it does find them.